# **Algorítmo de ranqueamento de páginas usado por websites de busca**

**O algorítmo de *Pagerank* é usado pelo Google e outros sites de busca para definir a relevância de cada uma das páginas web.**

****

**O algorítmo analisa as relações entre as páginas e de acordo com essas relações, é defininida a relevância de cada um dos sites.**

**Para entender melhor o fucionamento do algoritmo, se um site A extremamente relevante, faz referência a somente um site B, B será extremamente relevante assim como A. No entanto, caso A tenha referências a mais sites, B já não vai mais ser tão relevante assim.**

**Definindo o site referenciador, e o site referenciado, quanto mais referências o referenciado sofre, mais relevante ele será, e quanto mais referências o seu referenciador fizer, menos relevante ele será.**

****

In [1]:
#Import necessários para o funcionamento do Pagerank
import numpy as np
import random
import time
from html.parser import HTMLParser
from urllib import parse
from urllib.parse import urlparse
from urllib.request import urlopen
import threading
from queue import Queue
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Para entender melhor o algorítmo

**Consideramos que usuários de uma rede web em um website A podem mudar para um website B caso A possua um hyperlink para B. Caso A possua um hyperlink para B e para C os usuários que mudarem de website possuem 50% de chance de ir para B e 50% de chance de ir para C.**

**Podemos montar uma matriz de transformação, que dado um vetor com o número de usuários em cada website, podemos determinar o número de usuário em cada website no instante seguinte. Para calcular a relevância de cada site, devemos aplicar essa matriz de transformação inúmeras vezes à um vetor com todos os seus valores iguais (consideramos que inicialmente existem a mesma quantidade de pessoas em todos os websites) e torcer para que essas transformações convirjam para algum vetor definido.**

---

**Analisando essas relações entre os websites podemos montar uma matriz quadrada que exemplifique isso, supondo que teremos a coluna do website A como a 1 e a coluna do website B como a 2. Vamos organizar a matriz de modo que na posição 1x2 tenhamos a probabilidade de um usuário em A se mover para B.**

**Quando temos o mesmo valor na coluna e linha (ou seja, 1x1, 2x2, 3x3, etc) sempre encontraremos o valor zero, uma vez que, um website que um usuario que é redirecionado para um website no qual ele já estava, tem o mesmo efeito de não mudar de website. Portanto, hyperlinks que referenciam o próprio website não são considerados.**

---

**Organizando as a matriz que vai determinar a nossa rede web dessa maneira, podemos garantir que ela será estocástica, garantindo que matriz será estocástica também podemos garantir que se essa matriz quadrada estocástica de transformação for aplicada a um vetor variadas vezes, o valor do vetor vai convergir para algum valor definido.**

# **Implementação**
**O algorítmo se baseia em transformar  as relações entre os websites em uma matriz de transformação, e aplica-la à uma matriz Nx1 (sendo N o número de websites) infinitas vezes, até o que o resultado dessa transformação tenda para alguma valor. Essa valor será a matriz de websites com os valores referentes à sua importância.**

No exemplo abaixo, usaremos a seguinte rede web simulada

![alt text](http://pi.math.cornell.edu/~mec/Winter2009/RalucaRemus/Lecture3/Images/graf1.PNG)

O grafo com as relações demonstrando, o quanto de importância cada website está passando para outro.

![alt text](http://pi.math.cornell.edu/~mec/Winter2009/RalucaRemus/Lecture3/Images/graf2.PNG)

A matriz de transformação que será aplicada é a seguinte:

![alt text](http://pi.math.cornell.edu/~mec/Winter2009/RalucaRemus/Lecture3/Images/matrix.gif)

In [2]:
def Pagerank(matrizTransform, erroMin, printV):
    v = np.ones((len(matrizTransform[:,1]), 1)) * (1/len(matrizTransform[:,1]))
    if(printV):
        print(v)
    contador = 0
    erro = 1
    tempo = time.time()
  
    while(erro > erroMin):
        contador += 1
        if(erro > np.linalg.norm(v - matrizTransform@v)):
            erro = np.linalg.norm(v - matrizTransform@v)
        v = matrizTransform@v
  
    if(printV):
        print(v)
    print("A matriz levou", contador, "iterações para convergir")
    print("O codigo levou", time.time() - tempo, "segundos para devolver um resultado")
    return v

A = np.column_stack([[0, 1/3, 1/3, 1/3],[0, 0, 1/2, 1/2], [1, 0, 0, 0], [1/2, 0, 1/2, 0]])
Pagerank(A, 0.00000000001, True)

[[0.25]
 [0.25]
 [0.25]
 [0.25]]
[[0.38709677]
 [0.12903226]
 [0.29032258]
 [0.19354839]]
A matriz levou 42 iterações para convergir
O codigo levou 0.0009613037109375 segundos para devolver um resultado


array([[0.38709677],
       [0.12903226],
       [0.29032258],
       [0.19354839]])

**Podemos ver o que acontece quando aplicamos o algoritmo à redes web ainda maiores e observar a diferença no tempo de execução e no número de iterações**

In [3]:
def GerarRedeWeb(size):
    A = np.empty(size)
    for x in range(A.shape[0]):
        for y in range(A.shape[1]):
            A[x][y] = 1 if random.randint(0,2) > 0.5 else 0

    A /= np.sum(A, axis=0)  
    return A


print("MATRIX 5X5:") #Matrix 5x5
A = GerarRedeWeb((5, 5))
Pagerank(A, 0.0000000000000001, False)

print("MATRIX 6X6:") #Matrix 6x6
A = GerarRedeWeb((6, 6))
Pagerank(A, 0.0000000000000001, False)

print("MATRIX 90X90:") #Matrix 90x90
A = GerarRedeWeb((90, 90))
Pagerank(A, 0.0000000000000001, False)

print("MATRIX 180X180:") #Matrix 180x180
A = GerarRedeWeb((180, 180))
Pagerank(A, 0.0000000000000001, False)

print("MATRIX 1000X1000: ") #Matrix 1000x1000
A = GerarRedeWeb((1000, 1000))
Pagerank(A, 0.0000000000000001, False)

print("MATRIX 5000X5000: ") #Matrix 5000x5000
A = GerarRedeWeb((5000, 5000))
Pagerank(A, 0.0000000000000001, False)


MATRIX 5X5:
A matriz levou 23 iterações para convergir
O codigo levou 0.0010008811950683594 segundos para devolver um resultado
MATRIX 6X6:
A matriz levou 41 iterações para convergir
O codigo levou 0.0009996891021728516 segundos para devolver um resultado
MATRIX 90X90:
A matriz levou 14 iterações para convergir
O codigo levou 0.0009999275207519531 segundos para devolver um resultado
MATRIX 180X180:
A matriz levou 12 iterações para convergir
O codigo levou 0.012001276016235352 segundos para devolver um resultado
MATRIX 1000X1000: 
A matriz levou 9 iterações para convergir
O codigo levou 0.004002571105957031 segundos para devolver um resultado
MATRIX 5000X5000: 
A matriz levou 8 iterações para convergir
O codigo levou 0.17598390579223633 segundos para devolver um resultado


array([[0.00019888],
       [0.00019922],
       [0.000198  ],
       ...,
       [0.00020114],
       [0.00019801],
       [0.00019844]])

**Dessa forma podemos perceber que o tamanho da nossa rede web é inversamente proporcional ao número de iterações e proporcional ao tempo tomado pelo algorítmo para gerar um resultado.**

# Websites sem hyperlinks

**Podemos ter problemas com websites que não fazem referência à nenhum outro.**

---

**Nesses casos, vamos ter uma coluna inteira da matriz de transformação igual a zero, dessa forma não podemos assegurar que as transformações irão convergir para algum valor, uma vez que a matriz não é mais estocástica**

****

**Esse problema é de simples resolução; basta transformar todos os zeros em uma coluna zerada por 1/n (sendo n o número de websites em uma rede), dessa forma, vamos ter o website sem referências, dividindo a sua importância por todos os outros websites.**






# Um Web Crawler
**As classes a seguir são definidas para se construir um web crawler, que vai funcionar para procurar por hiperlinks em um determinado conjunto de páginas, e assim dessa forma poder se estabeler o ranking de relevância entre as páginas exploradas por esse Web Crawler.**

---

**A classe LinkFinder procura por hiperlinks em um determinado arquivo html e os retorna.**

---

**A classes Spider é o Web Crawler por si, ela possui as variáveis globais 'queue' e 'crawled', que representam respectivamente os sites na fila para serem *crawled*, enquanto a variável 'crawled' armazenará os websites que já foram explorados.
Essas variáveis são atualizadas a cada página explorada.
A classe Spider será chamada por threads para otimização de tempo, dessa forma, é necessário que as variáveis 'queue' e 'crawled' sejam globais da classe para que todas as threads tenham acesso a elas.**

---

**É importante notar que também é passado para a classe Spider o domínio dos websites, que serão explorados, para que websites fora daquele domínio não sejam explorados, fazendo assim com que o algorítmo, eventualmente, termine.**

---

**Usaremos esse crawler para popular um dicionario, cujas as chaves serão cada um dos url's e os valores de cada chave serão um set() com todos os url's ao quais a url da chave faz referência**

In [4]:
class LinkFinder(HTMLParser):
    def __init__(self, base_url, page_url):
        super().__init__()
        self.base_url = base_url
        self.page_url = page_url
        self.links = set()
        
    def handle_starttag(self, tag, attrs):
        if tag == 'a':
            for (attribute, value) in attrs:
                if attribute == 'href':
                    url = parse.urljoin(self.base_url, value)
                    self.links.add(url)
        
    def page_links(self):
        return self.links
        
    def error(self, message):
        pass
    

In [5]:
class Spider:
    project_name = ''
    base_url = ''
    domain_name = ''
    queue = set()
    crawled = set()
    
    def __init__(self, project_name, base_url, domain_name):
        Spider.project_name = project_name
        Spider.base_url = base_url
        Spider.domain_name = domain_name
        Spider.queue.add(base_url)
        
    def crawl_page(thread_name, page_url, page_dict):
        if page_url not in Spider.crawled:
            print(thread_name + ' crawling ' + page_url)
            Spider.add_links_to_queue(Spider.gather_links(page_url, page_dict))
            if page_url in Spider.queue:
                Spider.queue.remove(page_url)
            Spider.crawled.add(page_url)
            
    def gather_links(page_url, page_dict):
        html_string = ''
        try:
            response = urlopen(page_url)
            html_bytes = response.read()
            html_string = html_bytes.decode("utf-8")
            
            finder = LinkFinder(Spider.base_url, page_url)
            finder.feed(html_string)
        except:
            return set()
        
        page_links = finder.page_links()
        page_dict[page_url] = page_links
        return page_links
    
    def add_links_to_queue(links):
        for url in links:
            if url in Spider.queue:
                continue
            if url in Spider.crawled:
                continue
            if Spider.domain_name not in url:
                continue
            Spider.queue.add(url)


# A execução

**Na seguinte célula, 8 threads são criadas e executam a classe Spider, junto com a fila de urls que será iterativamente atualizada para cada thread pegar uma nova url e explorará.**

In [6]:
PROJECT_NAME = 'pagerank'
HOMEPAGE = 'https://www.dcc.ufrj.br/'
DOMAIN_NAME = 'dcc.ufrj.br'
NUMBER_OF_THREADS = 8

queue = Queue()
Spider(PROJECT_NAME, HOMEPAGE, DOMAIN_NAME)
page_dict = {}

def create_workers():
    for _ in range(NUMBER_OF_THREADS):
        t = threading.Thread(target=work)
        t.daemon = True
        t.start()
        
def work():
    while True:
        url = queue.get()
        Spider.crawl_page(threading.current_thread().name, url, page_dict)
        queue.task_done()
    
def create_jobs():
    for link in Spider.queue:
        queue.put(link)
    queue.join()
    crawl()
    
def crawl():
    queued_links = Spider.queue
    if len(queued_links) > 0:
        create_jobs()
        
        
create_workers()
crawl()

Thread-6 crawling https://www.dcc.ufrj.br/
Thread-12 crawling https://www.dcc.ufrj.br/grupos-de-extensao
Thread-8 crawling https://www.dcc.ufrj.br/images/JA_University/DCC//lep1-2.JPG
Thread-10 crawling https://www.dcc.ufrj.br/informacoes/noticias/430-sbsc-2021
Thread-13 crawling http://dcc.ufrj.br/index.php?option=com_content&view=article&id=422
Thread-9 crawling https://www.dcc.ufrj.br/informacoes/corpo-docente
Thread-7 crawling https://www.dcc.ufrj.br/images/JA_University/DCC//dcc2.JPG
Thread-6 crawling https://www.dcc.ufrj.br/ensino/geral/calendario-academico
Thread-11 crawling https://www.dcc.ufrj.br/images/JA_University/DCC//openhouse2.JPG
Thread-11 crawling https://www.dcc.ufrj.br/ofertas/oferta-de-monitoria
Thread-7 crawling https://www.dcc.ufrj.br/2014-11-18-16-55-42
Thread-8 crawling http://www.dcc.ufrj.br/~monitoriadcc/
Thread-8 crawling https://www.dcc.ufrj.br/informacoes/noticias/429-aulas-remotas-do-bacharelado-em-ciencia-da-computacao-2020-2
Thread-13 crawling https://ww

Thread-11 crawling http://gris.dcc.ufrj.br/
Thread-6 crawling https://www.dcc.ufrj.br/informacoes/noticias?start=50
Thread-8 crawling https://dcc.ufrj.br/attachments/article/58/pfc_1inscricao.pdf
Thread-12 crawling https://cainfo.dcc.ufrj.br/
Thread-9 crawling https://dcc.ufrj.br/attachments/article/66/Programa de estágio- agosto 2019.pdf
Thread-9 crawling https://dcc.ufrj.br/attachments/article/58/projeto.doc
Thread-10 crawling https://cainfo.dcc.ufrj.br/infraestrutura/
Thread-9 crawling https://www.dcc.ufrj.br/ensino/graduacao/calendario-de-provas?tmpl=component&print=1&page=
Thread-12 crawling https://www.dcc.ufrj.br/~marcellogt
Thread-8 crawling https://www.dcc.ufrj.br/#InfoGeral
Thread-10 crawling https://www.dcc.ufrj.br/component/k2/qual-a-diferenca-entre-inscricao-alteracao-e-trancamento-de-disciplinas
Thread-6 crawling https://cainfo.dcc.ufrj.br/comunicacao
Thread-12 crawling https://cainfo.dcc.ufrj.br/#material
Thread-9 crawling http://www.dcc.ufrj.br/~silvana
Thread-12 crawl

Thread-8 crawling http://www.dcc.ufrj.br/~sadoc/machinelearning
Thread-10 crawling http://www.dcc.ufrj.br/~rincon/Programas_Livro/solver.h
Thread-10 crawling https://www.dcc.ufrj.br/img/PortoGalinhas.jpg
Thread-13 crawling https://www.dcc.ufrj.br/#disciplina/44/Metodos-Numericos-I
Thread-8 crawling https://www.dcc.ufrj.br/#Disciplinas
Thread-9 crawling http://www.dcc.ufrj.br/~rincon/Disciplinas/Calculo Numerico/sistema_linear.pdfThread-7 crawling https://www.dcc.ufrj.br/comp2

Thread-9 crawling https://www.dcc.ufrj.br/img/toy.jpg
Thread-11 crawling http://www.dcc.ufrj.br/~gabriel/unixes.php
Thread-11 crawling https://www.dcc.ufrj.br/#disciplina/57/Recuperacao-da-Informacao
Thread-12 crawling https://www.dcc.ufrj.br/#professor/81/Paulo-Goldfeld
Thread-6 crawling https://devmob.dcc.ufrj.br/index.php/projetos/
Thread-10 crawling https://www.dcc.ufrj.br/#disciplina/24/Fundamentos-da-Engenharia-de-Software
Thread-8 crawling https://www.dcc.ufrj.br/#professor/26/Josefino-Cabral-Melo-Lima
Thr

Thread-13 crawling https://www.dcc.ufrj.br/#disciplina/20/Computacao-Concorrente
Thread-6 crawling https://www.dcc.ufrj.br/#disciplina/43/Analise-Numerica-de-Equacoes-Diferenciais-Parciais-I
Thread-10 crawling https://www.dcc.ufrj.br/#professor/77/Luca-Roberto-Augusto-Moriconi
Thread-11 crawling https://www.dcc.ufrj.br/#professor/66/Joaquim-Lopes-Neto
Thread-7 crawling https://www.dcc.ufrj.br/#professor/85/Ildeu-De-Castro-Moreira
Thread-12 crawling https://www.dcc.ufrj.br/#disciplina/26/Computadores-e-Sociedade
Thread-8 crawling https://www.dcc.ufrj.br/#disciplina/37/Laboratorio-de-Sistemas-Digitais
Thread-9 crawling https://www.dcc.ufrj.br/lp
Thread-13 crawling https://www.dcc.ufrj.br/informacoes/noticias/369-concursos-publicos-para-docentes-na-ufrj
Thread-6 crawling https://www.dcc.ufrj.br/informacoes/noticias/395-calendario-do-processo-seletivo-para-professor-substituto-programacao-de-computadores
Thread-10 crawling https://www.dcc.ufrj.br/#professor/21/Geraldo-Bonorino-Xexeo
Thread

Thread-11 crawling http://www.dcc.ufrj.br/~rincon/Disciplinas/calcnum.pdf
Thread-10 crawling https://www.dcc.ufrj.br/informacoes/noticias/386-insurtech-innovation-program
Thread-11 crawling http://www.dcc.ufrj.br/~rincon/Disciplinas/Metodos Numericos/CALENDARIO_PROVAS_2019_2.xls
Thread-11 crawling https://www.dcc.ufrj.br/index_pt-BR.htm
Thread-8 crawling https://www.dcc.ufrj.br/component/mailto/?tmpl=component&template=ja_university&link=9c3778045d24b4b94bb2e744152fa9dcab4e701d
Thread-6 crawling https://www.dcc.ufrj.br/#disciplina/32/Avaliacao-e-Desempenho
Thread-9 crawling https://www.dcc.ufrj.br/comp20131
Thread-13 crawling https://www.dcc.ufrj.br/#professor/63/Alexis-Hernandez
Thread-12 crawling https://www.dcc.ufrj.br/#professor/10/Claudson-Ferreira-Bornstein
Thread-7 crawling https://www.dcc.ufrj.br/publicacoes.html
Thread-11 crawling https://www.dcc.ufrj.br/#professor/91/Nei-Rocha
Thread-8 crawling https://www.dcc.ufrj.br/informacoes/noticias/384-alunos-do-bcc-premiados-na-gameth

Thread-9 crawling https://www.dcc.ufrj.br/#professor/52/Silvana-Rossetto
Thread-7 crawling https://www.dcc.ufrj.br/informacoes/noticias/397-professora-do-dcc-recebe-premio-marie-curie
Thread-10 crawling https://www.dcc.ufrj.br/lua
Thread-11 crawling https://www.dcc.ufrj.br/component/k2/duvidas#startOfPageId200
Thread-8 crawling https://www.dcc.ufrj.br/Livros.html
Thread-13 crawling https://www.dcc.ufrj.br/informacoes/noticias/385-provas-de-mudanca-de-curso-transferencia-externa-e-isencao-de-concurso-de-acesso-2019-1
Thread-9 crawling https://www.dcc.ufrj.br/#professor/3/Adriano-Joaquim-de-Oliveira-Cruz
Thread-11 crawling https://www.dcc.ufrj.br/#professor/29/Joao-Lauro-Dornelles-FacoThread-10 crawling https://www.dcc.ufrj.br/#disciplina/27/Arquitetura-de-Computadores-IThread-8 crawling https://www.dcc.ufrj.br/component/k2/duvidas?tmpl=component&print=1


Thread-7 crawling https://www.dcc.ufrj.br/informacoes/noticias/415-tcc-16-06-2020
Thread-12 crawling https://www.dcc.ufrj.br/#discipl

Thread-11 crawling https://www.dcc.ufrj.br/informacoes/noticias/403-prof-paulo-h-de-aguiar-rodrigues-condecorado-com-o-premio-destaques-em-governanca-da-internet-no-brasil
Thread-7 crawling https://www.dcc.ufrj.br/Disciplinas.html
Thread-12 crawling https://www.dcc.ufrj.br/comp20112
Thread-10 crawling http://www.dcc.ufrj.br/~rincon/Disciplinas/Metodos Numericos/ListaExercicios1.pdf
Thread-10 crawling https://www.dcc.ufrj.br/#disciplinas
Thread-13 crawling http://www.dcc.ufrj.br/~rincon/Disciplinas/Calculo Numerico/TRABALHOSUGESTAO.pdf
Thread-13 crawling https://www.dcc.ufrj.br/#professor/43/Nelson-Quilula-Vasconcelos
Thread-9 crawling https://www.dcc.ufrj.br/#professor/34/Marcello-Goulart-Teixeira
Thread-8 crawling http://www.dcc.ufrj.br/~rincon/Disciplinas/Metodos Numericos/TRABALHOSUGESTAO.pdf
Thread-8 crawling http://www.dcc.ufrj.br/%7Egabriel/arqcomp/index.php
Thread-8 crawling https://www.dcc.ufrj.br/component/mailto/?tmpl=component&template=ja_university&link=91c91bb9ecba57cb5ace

Thread-8 crawling https://www.dcc.ufrj.br/informacoes/noticias/396-processo-seletivo-para-professor-substituto-2?tmpl=component&print=1&page=
Thread-7 crawling https://www.dcc.ufrj.br/apostila.pdf
Thread-8 crawling https://www.dcc.ufrj.br/component/mailto/?tmpl=component&template=ja_university&link=19916fbc6245e4435ad041174ea7a94d0cae0638
Thread-7 crawling http://www.dcc.ufrj.br/attachments/article/377/Computação%20I%20(CC)%20-%20MAB120.pdf
Thread-7 crawling https://www.dcc.ufrj.br/informacoes/noticias/403-prof-paulo-h-de-aguiar-rodrigues-condecorado-com-o-premio-destaques-em-governanca-da-internet-no-brasil?tmpl=component&print=1&page=
Thread-8 crawling https://www.dcc.ufrj.br/informacoes/noticias/366-resultados-da-final-brasileira-da-maratona-de-programacao-2017?tmpl=component&print=1&page=
Thread-7 crawling https://www.dcc.ufrj.br/Netuno.jpg
Thread-8 crawling http://www.dcc.ufrj.br/~gabriel/boletim/boletim15.pdf
Thread-7 crawling https://www.dcc.ufrj.br/component/k2/content/3-grad

Thread-9 crawling https://www.dcc.ufrj.br/component/mailto/?tmpl=component&template=ja_university&link=3858ddd7d596a1bac0d484ec7ac5eb38ae96f111
Thread-12 crawling http://www.dcc.ufrj.br/~sadoc/ad20152/mab515-ementa.pdf
Thread-12 crawling https://www.dcc.ufrj.br/informacoes/noticias/400-contato-da-biblioteca-nce?tmpl=component&print=1&page=
Thread-7 crawling https://www.dcc.ufrj.br/component/mailto/?tmpl=component&template=ja_university&link=47c1c41b4ae3ca9fc1fb747dc8d45d03c61ad76c
Thread-9 crawling https://www.dcc.ufrj.br/DSC00735.JPG
Thread-7 crawling http://www.dcc.ufrj.br/~gabriel/boletim/boletim10.pdf
Thread-9 crawling https://www.dcc.ufrj.br/informacoes/noticias/408-palestra-eficiencia-energetica-com-uso-de-algoritmos-evolutivos-hibridos-prof-carolina-gil-marcelino-05-06-2020-14-00?tmpl=component&print=1&page=
Thread-12 crawling https://www.dcc.ufrj.br/component/mailto/?tmpl=component&template=ja_university&link=1f4a8090cf22007d7dc089f899ffbfadab6dc159
Thread-9 crawling https://ww

Thread-6 crawling https://www.dcc.ufrj.br/componentes/public/corpo-docente?page=4
Thread-13 crawling https://www.dcc.ufrj.br/componentes/public/professor/show/189
Thread-10 crawling http://dcc.ufrj.br/~sadoc
Thread-11 crawling https://accounts.google.com/AccountChooser?continue=https%3A%2F%2Fsites.google.com%2Fdcc.ufrj.br%2Fmitre&dsh=2281462176439503292
Thread-8 crawling https://www.dcc.ufrj.br/componentes/public/corpo-docente?page=3
Thread-7 crawling https://www.dcc.ufrj.br/dcc.ufrj.br/~aloisiopina
Thread-9 crawling https://www.dcc.ufrj.br/componentes/public/professor/show/177
Thread-12 crawling https://www.dcc.ufrj.br/componentes/public/professor/show/26
Thread-6 crawling http://dcc.ufrj.br/~carla
Thread-13 crawling http://dcc.ufrj.br/~avivacqua
Thread-12 crawling https://www.dcc.ufrj.br/componentes/public/professor/show/163
Thread-8 crawling https://www.dcc.ufrj.br/componentes/public/professor/show/3
Thread-9 crawling https://www.dcc.ufrj.br/componentes/public/professor/show/149
Thr

# Gerando a matriz de relações

**Usando o dicionário que foi construido pelas threads, uma matriz da forma que é aceita pelo PageRank é construida e printada ao final da execução da seguinte célula.**

---

**No seguinte exemplo a matriz de relações a partir da url 'https://www.dcc.ufrj.br/' de todas as páginas no domínio 'dcc.ufrj.br', que foram os parâmetros passados ao crawler na célula anterior.**

In [7]:
size = (len(page_dict.keys()), len(page_dict.keys()))
A = np.empty(size)
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        A[i][j] = 1 if list(page_dict.keys())[j] in page_dict[list(page_dict.keys())[i]] else 0 

A /= np.sum(A, axis=0)
print(A)

[[0.0018622  0.00239234 0.00239234 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.0018622  0.00239234 0.00239234 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.0018622  0.00239234 0.00239234 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.5       ]]


# Aplicando o PageRank

**Aplicando o PageRank à matriz gerada na célula anterior, temos o array de relevância de cada uma das páginas do domínio 'dcc.ufrj.br'.**

---

**Na célula após a seguinte é printada a url de maior relevância.**

In [8]:
arrayPagerank = Pagerank(A, 0.0000000000000001, False)

A matriz levou 344 iterações para convergir
O codigo levou 0.03799772262573242 segundos para devolver um resultado


In [9]:
list(page_dict.keys())[np.argmax(arrayPagerank)]

'https://infoprovas.dcc.ufrj.br/'

*Feito por Ronald Albert de Araujo Junior e Diogo Rocco Crivaro Nunes*

*Alunos de Ciência da Computação na Universidade Federal do Rio De Janeiro*

*DRE: 118021192*